## InBody Graph

#### WT: 체중 kg
#### SMM: 골격근량 kg
#### BFM: 체지방  kg
#### PFM: 체지방률 %

In [ ]:
import sys
from utils import crawling, clean_strings, filters, seperate_value

results_fetch = list(filter(
        lambda s: s.startswith('#'),
        filters(
            crawling(
                f'https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/InBody.md',
                err=lambda e: None),
            lambda lines: [] if lines is None else lines.split('\n'),
            lambda s: clean_strings(s, str.strip))))

if len(results_fetch) == 0:
    sys.exit(0)

print(results_fetch)

In [ ]:
results_arrange = []

for record in results_fetch:
    if record.startswith('####'):
        item, value = (s.strip() for s in record.replace('####', '').strip().split(':'))
        results_arrange.append((date, item, *seperate_value(value)))
    elif record.startswith('##'):
        date = record.replace('##', '').strip()

print(results_arrange)

In [ ]:
from datetime import datetime

dataset_x, dataset_y, d, key = [], {}, '', ''
for result in results_arrange:
    if d != result[0]:
        d = result[0]
        dataset_x.append(datetime.strptime(d, '%Y.%m.%d'))

    if key != result[1]:
        key = result[1]
        if key not in dataset_y:
            dataset_y.update({key: {'unit': result[3], 'values': []}})
        dataset_y[key]['values'].append(result[2])

print(dataset_x)
print(dataset_y)

In [ ]:
from matplotlib.pyplot import figure
from itertools import cycle
from matplotlib import pyplot as plt

dataset_x_days = [(x - dataset_x[0]).days for x in dataset_x]
dataset_x_fdates = [x.strftime('%m.%d') for x in dataset_x]

legendhandles = []
cycol = cycle('bgrcmk')

fig = plt.figure(figsize=(30, 10))
spec = plt.GridSpec(1, 1)
subplot = fig.add_subplot(spec[0, 0])

subplot.set_yticks([])
subplot.set_xticks(dataset_x_days)
subplot.set_xticklabels(dataset_x_fdates, fontsize=9)

twinx = None
for key in dataset_y.keys():
    twinx = (subplot if twinx is None else twinx).twinx()

    twinx.set_yticks([])
    line, = twinx.plot(dataset_x_days, dataset_y[key]['values'], 'o-', color=next(cycol), label=f"{key}({dataset_y[key]['unit']})")

    for idx, yv in enumerate(dataset_y[key]['values']):
        twinx.annotate(str(yv), xy=(dataset_x_days[idx], yv), xytext=(-10, 5), textcoords='offset points')

    legendhandles.append(line)

plt.legend(handles=legendhandles, loc='lower center')
plt.show()